### Import Libraries

In [ ]:
!pip install torch_geometric
!pip install networkit
!apt install libgraphviz-dev
!pip install pygraphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 2.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libgvc6-plugins-gtk
  librsvg2-common libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk librsvg2-common libxdot4
0 upgraded, 9 newly installed, 0 to remove and 45 not upgraded.
Need to get 2,433 kB of archives.
After this operation, 7,694 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgtk2.0-common all 2.24.33-2ubuntu2 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgtk2.0-0 amd64 2.24.33-2ubuntu2 [2,037 

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn

import json
import os
import networkx as nx
import networkit as nk
from networkit import vizbridges

from tqdm import tqdm
import ast
import matplotlib.pyplot as plt
import re
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import ndcg_score

from torch_geometric.data import Data
from torch_geometric.utils import subgraph
from torch_geometric.nn import GATv2Conv, GCNConv

import copy
from typing import Callable, Tuple

import torch
from torch import Tensor
from torch.nn import Module, Parameter

from torch_geometric.nn.inits import reset, uniform
import random

In [ ]:
torch.manual_seed(200)
np.random.seed(200)

### Util functions

In [ ]:
def load_dataset_timestamp( n_users, n_context, seq_len):
    act_list = []
    time_list = []
    user_list = []

    max_timestamp = -1.0
    min_timestamp = float('inf')

    with open('gowalla_user_activity.txt', 'r') as raw_file:
        for line in raw_file:
            t_item_list = []
            t_time_list = []
            user = int(line.split(':')[0])
            entries = line.split()[1:]
            for a_entry in entries:
                item, time_stamp = a_entry.split(':')
                t_item_list.append(int(item.strip()))
                t_time_list.append(int(time_stamp.strip()))

                if min_timestamp > int(time_stamp.strip()):
                    min_timestamp = int(time_stamp.strip())
                if max_timestamp < int(time_stamp.strip()):
                    max_timestamp = int(time_stamp.strip())

            act_list.append(t_item_list[0: seq_len])
            time_list.append(t_time_list[0: seq_len])
            user_list.append(user)

    new_time_list = []
    num_bins = 0

    times_bins = np.linspace(min_timestamp, max_timestamp + 1, num=num_bins, dtype=np.int32)
    for a_time_list in time_list:
        temp_time_list = (np.digitize(np.asarray(a_time_list), times_bins) - 1).tolist()
        new_time_list.append(temp_time_list)

    all_examples = {}
    for i in range(0, len(act_list)):

        train_act_seq = act_list[i][:-2]

        train_time_seq = new_time_list[i][:-2]

        train_act_label = act_list[i][-2]
        train_time_label = new_time_list[i][-2]

        test_act_seq = act_list[i][1:-1]
        test_time_seq = new_time_list[i][1:-1]

        test_act_label = act_list[i][-1]
        test_time_label = new_time_list[i][-1]

        entry = {
            'train_act_seq': train_act_seq,
            'train_time_seq': train_time_seq,
            'train_act_label': train_act_label,
            'train_time_label': train_time_label,
            'test_act_seq': test_act_seq,
            'test_time_seq': test_time_seq,
            'test_act_label': test_act_label,
            'test_time_label': test_time_label,
            'seq_len': len(train_act_seq),
            'user': user_list[i]
        }

        all_examples[user_list[i]] = entry

    return all_examples

### Metrics

In [ ]:
def print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100):
    print(f'hits@1: {hits1:.6f}, hits@5: {hits5:.6f}, hits@10: {hits10:.6f}, hits@20: {hits20:.6f}')
    print(f'hits@50: {hits50:.6f}, hits@100: {hits100:.6f}')
    print(f'map@1: {map1:.6f}, map@5: {map5:.6f}, map@10: {map10:.6f}, map@20: {map20:.6f}')
    print(f'map@50: {map50:.6f}, map@100: {map100:.6f}')
    print(f'ndcg@1: {ndcg1:.6f}, ndcg@5: {ndcg5:.6f}, ndcg@10: {ndcg10:.6f}, ndcg@20: {ndcg20:.6f}')
    print(f'ndcg@50: {ndcg50:.6f}, ndcg@100: {ndcg100:.6f}')

In [ ]:
def apk(actual, predicted, k=10):

    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(y_prob, y, k=10):

    predicted = [np.argsort(p_)[-k:][::-1] for p_ in y_prob]
    actual = [[y_] for y_ in y]
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])


def hits_k(y_prob, y, k=10):
    acc = []
    for p_, y_ in zip(y_prob, y):
        top_k = p_.argsort()[-k:][::-1]
        acc += [1. if y_ in top_k else 0.]
    return sum(acc) / len(acc)

In [ ]:
def get_metrics_(probs, labels_batch, test_one_hot):
    hits1 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=1, labels = classes)
    hits5 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=5, labels = classes)
    hits10 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=10, labels = classes)
    hits20 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=20, labels = classes)
    hits50= top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=50, labels = classes)
    hits100 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=100, labels = classes)

    map1 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=1)
    map5 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=5)
    map10 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=10)
    map20 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=20)
    map50 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=50)
    map100 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=100)

    ndcg1 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=1)
    ndcg5 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=5)
    ndcg10 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=10)
    ndcg20 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=20)
    ndcg50 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=50)
    ndcg100 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=100)
    return hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100

### Graph construction

In [ ]:
users = load_dataset_timestamp(20001, 128, 30)

In [ ]:
friends = pd.read_csv('gowalla_edges.csv')

In [ ]:
Gcl = nx.Graph()
for i in range(len(users)):
  if users[i]['user'] not in Gcl.nodes():
      Gcl.add_node(users[i]['user'], weight=10, color='seagreen')
for i in range(len(friends)):
  Gcl.add_edge(friends.loc[i, '1st friend'], friends.loc[i, '2nd friend'], color = 'blue')



nodes = list(Gcl.nodes())
node_colors = [Gcl.nodes[node]['color'] for node in Gcl.nodes]
node_weights = [Gcl.nodes[node]['weight'] for node in Gcl.nodes]
edge_colors = [Gcl.edges[edge]['color'] for edge in Gcl.edges]

plt.figure(figsize=(12, 8))
pos = nx.drawing.nx_agraph.graphviz_layout(Gcl, prog='neato')
nx.draw_networkx(Gcl, pos=pos, with_labels=False, node_size=node_weights, node_color=node_colors, edge_color=edge_colors, width=1)

In [ ]:
Gcl = nx.Graph()
for i in range(len(users)):
  if users[i]['user'] not in Gcl.nodes():
      Gcl.add_node(users[i]['user'], weight=10, color='seagreen')
for i in range(len(friends)):
  Gcl.add_edge(friends.loc[i, '1st friend'], friends.loc[i, '2nd friend'], color = 'blue')


nodes = list(Gcl.nodes())
node_colors = [Gcl.nodes[node]['color'] for node in Gcl.nodes]
node_weights = [Gcl.nodes[node]['weight'] for node in Gcl.nodes]
edge_colors = [Gcl.edges[edge]['color'] for edge in Gcl.edges]

In [ ]:
graphid2pid = {}
pid2graphid = {}

s = 0
for node_id in Gcl.nodes():
    pid2graphid[node_id] = s
    graphid2pid[s] = node_id
    s += 1

graph_edges = []
for (u, v) in Gcl.edges():
    graph_edges.append((pid2graphid[u], pid2graphid[v]))

### Graph characteristics

In [ ]:
nkg = nk.Graph(n=len(pid2graphid), weighted=True, directed=False, edgesIndexed=True)
for (u, v) in graph_edges:
    nkg.addEdge(u, v)

In [ ]:
nk.overview(nkg)

Network Properties:
nodes, edges			20001, 167752
directed?			False
weighted?			True
isolated nodes			275
self-loops			0
density				0.000839
clustering coefficient		0.294269
min/max/avg degree		0, 6171, 16.774361
degree assortativity		-0.064750
number of connected components	290
size of largest component	19698 (98.49 %)


### User dataset construction

In [ ]:
class UserInfoDataset():
  def __init__(self, data, max_len):
    self.data = data
    self.max_len = max_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):

    user = self.data[idx]
    seq_len = user['seq_len']

    tr_act_seq = np.zeros((self.max_len,)).astype('int32')
    tr_act_seq[:seq_len] = np.array(user['train_act_seq'])
    tr_act_seq = np.transpose(tr_act_seq)

    tr_time_seq = np.zeros((self.max_len,)).astype('int32')
    tr_time_seq[:seq_len] = user['train_time_seq']
    tr_time_seq = np.transpose(tr_time_seq)

    t_act_seq = np.zeros((self.max_len, )).astype('int32')
    t_act_seq[:seq_len] = user['test_act_seq']
    t_act_seq = np.transpose(t_act_seq)

    t_time_seq = np.zeros((self.max_len, )).astype('int32')
    t_time_seq[:seq_len] = user['test_time_seq']
    t_time_seq = np.transpose(t_time_seq)


    return user['user'], tr_act_seq, \
    tr_time_seq, user['train_act_label'], \
    user['train_time_label'], t_act_seq, \
    t_time_seq, user['test_act_label'], \
    user['test_time_label'], user['seq_len']

In [ ]:
from torch.utils.data import Subset
from torch.utils.data import DataLoader

user_dataset = UserInfoDataset(users, 30)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_dataloader = DataLoader(user_dataset, batch_size = 64, shuffle = True)
user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

item_emb  = nn.init.xavier_uniform_(torch.empty(186, 128))

## Pre-training experiments

In [ ]:
rnn = nn.RNN(128, 128, batch_first = True)
norm = nn.BatchNorm1d(128)

### Getting initial user representations

In [ ]:
user_representations = {}
for users, train_input, train_time, train_label, train_time_label, test_input, test_time, test_label, test_time_label, seq_len in user_dataloader:
      comb_input = np.concatenate([np.expand_dims(train_input, axis=-1),
                                                np.expand_dims(train_time, axis=-1)], axis=2)
      model_input = comb_input
      model_output = train_label
      rnn_input_emb = item_emb[model_input[:, :, 0]]
      test_comb_input = np.concatenate([np.expand_dims(test_input, axis=-1),
                                                np.expand_dims(test_time, axis=-1)], axis=2)
      test_model_input = test_comb_input
      test_rnn_input_emb = item_emb[test_model_input[:, :, 0]]
      x, h = rnn(rnn_input_emb)
      test_x, test_h = rnn(test_rnn_input_emb)
      hx = torch.zeros(x.shape[0], x.shape[2])#.to(device)
      for i in range(hx.shape[0]):
        hx[i] = x[i][seq_len[i] - 1]
      user_representation = norm(hx)
      test_hx = torch.zeros(test_x.shape[0], test_x.shape[2])
      for i in range(test_hx.shape[0]):
        test_hx[i] = test_x[i][seq_len[i] - 1]
      test_user_representation = norm(test_hx)
      for i in range(len(users)):
        user_representations[users[i].item()] = (users[i].item(), user_representation[i], train_label[i].item(), seq_len[i].item(), test_user_representation[i], test_label[i].item())

In [ ]:
user_ids = []
features = []
train_labels = []
seq_lens = []
test_features = []
test_labels = []
for graphid in Gcl.nodes():
    pid = graphid2pid[graphid]
    if graphid in user_representations.keys():
      user_ids.append(user_representations[graphid][0])
      features.append(user_representations[graphid][1].detach().numpy())
      train_labels.append(user_representations[graphid][2])
      test_features.append(user_representations[graphid][4].detach().numpy())
      seq_lens.append(user_representations[graphid][3])
      test_labels.append(user_representations[graphid][5])
user_ids = np.array(user_ids)
features = np.array(features)
train_labels = np.array(train_labels)
test_features = np.array(test_features)
seq_lens = np.array(seq_lens)
test_labels = np.array(test_labels)

In [ ]:
dataset = Data(x=torch.tensor(features, dtype=torch.float), edge_index=torch.tensor(np.array(graph_edges).T, dtype=torch.int64))

In [ ]:
friends

,Unnamed: 0,1st friend,2nd friend
0,0,0,1
1,1,0,2
2,2,0,3
3,3,0,4
4,4,0,5
...,...,...,...
335499,448763,20000,2892
335500,448764,20000,2937
335501,448765,20000,2940
335502,448766,20000,4375


In [ ]:
dataset.user_ids = torch.tensor(user_ids, dtype=torch.int64)
dataset.train_labels = torch.tensor(train_labels, dtype=torch.int64)
dataset.seq_lens = torch.tensor(seq_lens, dtype=torch.int64)
dataset.test_labels = torch.tensor(test_labels, dtype=torch.int64)
dataset.test_features = torch.tensor(test_features, dtype=torch.float)

In [ ]:
dataset.edge_index.shape

torch.Size([2, 167752])

In [ ]:
print(len(features))

20001


### Deep Graph Infomax model

In [ ]:
eps = 1e-15
class DeepGraphInfomax(torch.nn.Module):
    def __init__(
        self,
        hidden_channels,
        encoder,
        summary,
        corruption,
    ):
        super().__init__()
        self.hidden_channels = hidden_channels
        self.encoder = encoder
        self.summary = summary
        self.corruption = corruption

        self.weight = Parameter(torch.empty(hidden_channels, hidden_channels))

        self.reset_parameters()

    def reset_parameters(self):
        reset(self.encoder)
        reset(self.summary)
        uniform(self.hidden_channels, self.weight)

    def forward(self, *args, **kwargs) -> Tuple[Tensor, Tensor, Tensor]:
        pos_z = self.encoder(*args, **kwargs)

        cor = self.corruption(*args, **kwargs)
        cor = cor if isinstance(cor, tuple) else (cor, )
        cor_args = cor[:len(args)]
        cor_kwargs = copy.copy(kwargs)
        for key, value in zip(kwargs.keys(), cor[len(args):]):
            cor_kwargs[key] = value

        neg_z = self.encoder(*cor_args, **cor_kwargs)

        summary = self.summary(pos_z, *args, **kwargs)

        return pos_z, neg_z, summary

    def discriminate(self, z: Tensor, summary: Tensor,
                     sigmoid: bool = True) -> Tensor:

        summary = summary.t() if summary.dim() > 1 else summary
        value = torch.matmul(z, torch.matmul(self.weight, summary))
        return torch.sigmoid(value) if sigmoid else value

    def loss(self, pos_z: Tensor, neg_z: Tensor, summary: Tensor) -> Tensor:
        pos_loss = -torch.log(
            self.discriminate(pos_z, summary, sigmoid=True) + eps).mean()
        neg_loss = -torch.log(1 -
                              self.discriminate(neg_z, summary, sigmoid=True) +
                              eps).mean()

        return pos_loss + neg_loss


    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({self.hidden_channels})'

In [ ]:
class Encoder(nn.Module):
    def __init__(self, interm_channels, hidden_channels, num_features):
        super().__init__()

        self.num_features = num_features
        self.rnn = nn.RNN(128, 128, batch_first = True)
        self.norm = nn.BatchNorm1d(128)
        self.conv1 = GATv2Conv(self.num_features, interm_channels)
        self.prelu1 = nn.PReLU(interm_channels)
        self.dropout = nn.Dropout(0.2)
        self.conv2 = GATv2Conv(interm_channels, hidden_channels)
        self.prelu2 = nn.PReLU(hidden_channels)
        self.conv3 = GATv2Conv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.prelu1(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.prelu2(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return x

### Augmentation construction helping functions

In [ ]:
def index_to_mask(index):

    index = index.view(-1)
    size = int(index.max()) + 1

    mask = index.new_zeros(20001, dtype=torch.bool)
    mask[index] = True
    return mask

In [ ]:
def map_index(indices, edge_index):
  dictionary = {}
  label = 0
  for ind in indices:
    dictionary[ind] = label
    label += 1
  for i in range(edge_index.shape[1]):
    # if edge_index[0][i] in dictionary.keys():
    edge_index[0][i] = dictionary[edge_index[0][i].item()]
    # if edge_index[1][i] in dictionary.keys():
    edge_index[1][i] = dictionary[edge_index[1][i].item()]
  return edge_index

In [ ]:
def construct_neighbors_dictionary(edge_index):
  dictionary = {}
  for i in range(edge_index.shape[1]):
    if edge_index[0][i].item() not in dictionary.keys():
      dictionary[edge_index[0][i].item()] = []
    dictionary[edge_index[0][i].item()].append(edge_index[1][i].item())
    if edge_index[1][i].item() not in dictionary.keys():
      dictionary[edge_index[1][i].item()] = []
    dictionary[edge_index[1][i].item()].append(edge_index[0][i].item())
  return dictionary

In [ ]:
neighbors = construct_neighbors_dictionary(dataset.edge_index)

### Augmentations

In [ ]:
def node_mix(x, edge_index):
    return x[torch.randperm(x.size(0))], edge_index

In [ ]:
def node_dropout(x, edge_index):
  indices = []
  deleted_indices = []
  for i in range(x.shape[0]):
    drop = random.random()
    if drop >= 0.3:
      indices.append(i)
    else:
      deleted_indices.append(i)
  x = torch.index_select(x, 0, torch.tensor(indices, dtype = torch.int32))
  node_mask = index_to_mask(torch.tensor(indices, dtype = torch.int32))
  edge_mask = node_mask[edge_index[0]] & node_mask[edge_index[1]]
  edge_index = edge_index[:, edge_mask]
  edge_index = map_index(indices, edge_index)

  return x, edge_index

In [ ]:
def random_walk(x, edge_index):
  node = 0
  rwalk = [node]
  walk_length = 10000
  for i in range(walk_length - 1):
    temp = neighbors[node]
    temp = list(set(temp) - set(rwalk))
    if len(temp) == 0:
      break
    new_node = random.choice(temp)
    rwalk.append(new_node)
    node = new_node
  x = torch.index_select(x, 0, torch.tensor(rwalk, dtype = torch.int32))
  node_mask = index_to_mask(torch.tensor(rwalk, dtype = torch.int32))
  edge_mask = node_mask[edge_index[0]] & node_mask[edge_index[1]]
  edge_index = edge_index[:, edge_mask]
  edge_index = map_index(rwalk, edge_index)

  return x, edge_index

In [ ]:
def edge_dropout(x, edge_index):
  indices = []
  for i in range(edge_index.shape[1]):
    drop = random.random()
    if drop >= 0.3:
      indices.append(i)
  edge_index = torch.index_select(edge_index, 1, torch.tensor(indices, dtype = torch.int32))
  return x, edge_index

### Link prediction head

In [ ]:
class LinkPredHead(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc_1 = nn.Linear(256, 1)

    def forward(self, x, edge_index):
        x_src, x_dst = x[edge_index[0]], x[edge_index[1]]
        # x = torch.cat((x_src, x_dst), dim = 1)
        # x = self.fc_1(x)
        return torch.sum(x_src * x_dst, dim=1)

### Next action prediction head

In [ ]:
class NextActionPredHead(nn.Module):
  def __init__(self):
    super(NextActionPredHead, self).__init__()
    self.fc1 = nn.Linear(128, num_classes)
    self.conv1 = GATv2Conv(128, 128)
    self.rnn = nn.RNN(128, 128, batch_first = True)

  def forward(self, x):
    x = self.fc1(x)
    return x

In [ ]:
def negative_sampling(edge_index, num_nodes):
    mask_1 = torch.rand(edge_index.size(1)) < 0.5
    mask_2 = ~mask_1

    neg_edge_index = edge_index.clone()
    neg_edge_index[0, mask_1] = torch.randint(num_nodes, (mask_1.sum(),))
    neg_edge_index[1, mask_2] = torch.randint(num_nodes, (mask_2.sum(),))

    return neg_edge_index

### User representations dataset

In [ ]:
class UserDataset():
    def __init__(self, user_representations):
        self.user_representations = user_representations

    def __len__(self):
        return len(self.user_representations)

    def __getitem__(self, idx):

        return self.user_representations[idx][0], self.user_representations[idx][1], self.user_representations[idx][2]

### Training functions

In [ ]:
def train_only_infomax(epoch):
    model.train()
    optimizer.zero_grad()
    pos_z, neg_z, summary = model(dataset.x, dataset.edge_index)
    loss_infomax = model.loss(pos_z, neg_z, summary)

    pos_out = linkmodel(pos_z, dataset.edge_index)
    neg_edge_index = negative_sampling(dataset.edge_index, dataset.num_nodes)
    neg_out = linkmodel(pos_z, neg_edge_index)

    out = torch.cat([pos_out, neg_out])
    gt = torch.cat([torch.ones_like(pos_out), torch.zeros_like(neg_out)])
    loss_link_pred = loss_bce(out, gt)

    loss =  loss_infomax
    loss.backward()
    optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss}')
    if epoch == num_epoch:
      return pos_z, neg_z, summary, loss.item()
    else:
      return loss.item()

In [ ]:
def train_infomax_linkpred(epoch):
    model.train()
    optimizer.zero_grad()
    pos_z, neg_z, summary = model(dataset.x, dataset.edge_index)
    loss_infomax = model.loss(pos_z, neg_z, summary)

    pos_out = linkmodel(pos_z, dataset.edge_index)
    neg_edge_index = negative_sampling(dataset.edge_index, dataset.num_nodes)
    neg_out = linkmodel(pos_z, neg_edge_index)

    out = torch.cat([pos_out, neg_out])
    gt = torch.cat([torch.ones_like(pos_out), torch.zeros_like(neg_out)])
    loss_link_pred = loss_bce(out, gt)
    loss =  2*loss_infomax + 0.5*loss_link_pred
    loss.backward()
    optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss}')
    if epoch == num_epoch:
      return pos_z, neg_z, summary, loss.item()
    else:
      return loss.item()

In [ ]:
def test(head_model):
  metrics_val = []
  head_model.eval()
  index = 0
  for users, vectors, labels in user_test_dataloader:
      test_probs = head_model(vectors)
      test_pred = torch.argmax(test_probs, axis = 1)#torch.Tensor([0]*20001)

      test_one_hot = torch.zeros(len(test_probs), num_classes)
      test_one_hot[torch.arange(len(test_one_hot)), labels] = 1


      hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 \
       = get_metrics_(test_probs, labels, test_one_hot)

      metrics_val.append([hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100])#, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100])

  mean = torch.Tensor(metrics_val).mean(axis=0)
  test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, \
  test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100 = mean

  return test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, \
  test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100

In [ ]:
def middle_test(model):
  hits_1 = []
  for users, vectors, labels in user_test_dataloader:

    test_probs = model(vectors)

    test_pred = torch.argmax(test_probs, axis = 1)

    test_one_hot = torch.zeros(len(test_probs), num_classes)
    test_one_hot[torch.arange(len(test_one_hot)), labels] = 1
    hits_1.append(top_k_accuracy_score(labels, test_probs.cpu().detach().numpy(), k=1, labels = classes))
  print(f'Hits@1: {torch.mean(torch.Tensor(hits_1))}')


In [ ]:
def train2task():
  loss_fn = nn.CrossEntropyLoss()
  head_model  = NextActionPredHead()

  head_model.train()
  optimizer = torch.optim.Adam(head_model.parameters(), lr = 0.0001)
  lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.97, last_epoch=-1)
  losses = []

  for i in range(200):
    for users, vectors, labels in user_train_dataloader:

        optimizer.zero_grad()

        probs = head_model(vectors)
        pred = torch.argmax(probs, axis = 1)
        one_hot = torch.zeros(len(probs), num_classes)
        one_hot[torch.arange(len(one_hot)), labels] = 1
        # print(one_hot.shape)
        # print(probs.shape)
        loss = loss_fn(probs, one_hot)

        losses.append(loss)
        loss.backward()
        optimizer.step()
    mean_loss = torch.Tensor(losses).mean(axis=0).item()
    print(f'Epoch: {i}, Loss: {mean_loss}')
    middle_test(head_model)

  hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = test(head_model)
  print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100)
  return head_model

### Node mix augmentation, only infomax

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=node_mix).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [ ]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_only_infomax(epoch)
  else:
    loss = train_only_infomax(epoch)
  loss_epoch.append(loss)

test_pos_z, test_neg_z, test_summary = model(dataset.test_features, dataset.edge_index)

  1%|          | 1/100 [00:07<12:41,  7.69s/it]

Epoch: 1, Loss: 2.0441513061523438


  2%|▏         | 2/100 [00:15<12:31,  7.67s/it]

Epoch: 2, Loss: 1.3706047534942627


  3%|▎         | 3/100 [00:21<11:28,  7.10s/it]

...

Epoch: 98, Loss: 0.20094075798988342


 99%|█████████▉| 99/100 [11:43<00:07,  7.05s/it]

Epoch: 99, Loss: 0.26364076137542725


100%|██████████| 100/100 [11:50<00:00,  7.10s/it]

Epoch: 100, Loss: 0.25947657227516174


In [ ]:
train_user_representations = {}
test_user_representations = {}
for i in range(pos_z.shape[0]):
  train_user_representations[i] = (dataset.user_ids[i], pos_z[i].detach(), dataset.train_labels[i])
for i in range(test_pos_z.shape[0]):
  test_user_representations[i] = (dataset.user_ids[i], test_pos_z[i].detach(), dataset.test_labels[i])

In [ ]:
user_dataset = UserDataset(train_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [ ]:
num_classes = 186
classes = np.arange(0, num_classes)

In [ ]:
head_model = train2task()
torch.save(head_model.state_dict(), 'node_mix_only_infomax_model.pth')

Epoch: 0, Loss: 4.948060035705566
Hits@1: 0.04782196879386902
Epoch: 1, Loss: 4.755845546722412
Hits@1: 0.054548460990190506
Epoch: 2, Loss: 4.627336502075195
Hits@1: 0.07984607666730881
Epoch: 3, Loss: 4.5391411781311035
Hits@1: 0.07826779037714005
Epoch: 4, Loss: 4.475954532623291
Hits@1: 0.07955297082662582
Epoch: 5, Loss: 4.428366661071777
Hits@1: 0.08254419267177582
Epoch: 6, Loss: 4.390832424163818
Hits@1: 0.08524230122566223
Epoch: 7, Loss: 4.360129356384277
Hits@1: 0.08921055495738983
Epoch: 8, Loss: 4.334252834320068
Hits@1: 0.09741011261940002
Epoch: 9, Loss: 4.311925888061523
Hits@1: 0.10312950611114502
Epoch: 10, Loss: 4.292300224304199
Hits@1: 0.10609066486358643
Epoch: 11, Loss: 4.274774074554443
Hits@1: 0.10858585685491562
Epoch: 12, Loss: 4.2589192390441895
Hits@1: 0.11032196879386902
Epoch: 13, Loss: 4.244439125061035
Hits@1: 0.1118401288986206
Epoch: 14, Loss: 4.231086254119873
Hits@1: 0.11354617774486542
Epoch: 15, Loss: 4.218681812286377
Hits@1: 0.11503426730632782


### Node dropout augmentation, only infomax

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=node_dropout).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [ ]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_only_infomax(epoch)
  else:
    loss = train_only_infomax(epoch)
  loss_epoch.append(loss)
test_pos_z, test_neg_z, test_summary = model(dataset.test_features, dataset.edge_index)

  1%|          | 1/100 [00:10<16:43, 10.14s/it]

Epoch: 1, Loss: 1.434377908706665


  2%|▏         | 2/100 [00:18<14:57,  9.16s/it]

Epoch: 2, Loss: 1.6858642101287842


  3%|▎         | 3/100 [00:25<13:03,  8.07s/it]

...

Epoch: 98, Loss: 0.5377374291419983


 99%|█████████▉| 99/100 [12:41<00:07,  7.80s/it]

Epoch: 99, Loss: 0.6868425607681274


100%|██████████| 100/100 [12:48<00:00,  7.69s/it]

Epoch: 100, Loss: 0.4218140244483948


In [ ]:
train_user_representations = {}
test_user_representations = {}
for i in range(pos_z.shape[0]):
  train_user_representations[i] = (dataset.user_ids[i], pos_z[i].detach(), dataset.train_labels[i])
for i in range(test_pos_z.shape[0]):
  test_user_representations[i] = (dataset.user_ids[i], test_pos_z[i].detach(), dataset.test_labels[i])

In [ ]:
user_dataset = UserDataset(train_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [ ]:
num_classes = 186
classes = np.arange(0, num_classes)

In [ ]:
head_model = train2task()
torch.save(head_model.state_dict(), 'node_dropout_only_infomax_model.pth')

Epoch: 0, Loss: 5.0717082023620605
Hits@1: 0.04041155427694321
Epoch: 1, Loss: 4.94442892074585
Hits@1: 0.05825366824865341
Epoch: 2, Loss: 4.8375325202941895
Hits@1: 0.08203312754631042
Epoch: 3, Loss: 4.747663974761963
Hits@1: 0.08848153799772263
Epoch: 4, Loss: 4.671762943267822
Hits@1: 0.09320887178182602
Epoch: 5, Loss: 4.607346534729004
Hits@1: 0.09345688670873642
Epoch: 6, Loss: 4.552304744720459
Hits@1: 0.09494498372077942
Epoch: 7, Loss: 4.5048627853393555
Hits@1: 0.09818422049283981
Epoch: 8, Loss: 4.463573455810547
Hits@1: 0.10140842944383621
Epoch: 9, Loss: 4.427262306213379
Hits@1: 0.1046476662158966
Epoch: 10, Loss: 4.394991874694824
Hits@1: 0.10905182361602783
Epoch: 11, Loss: 4.36603307723999
Hits@1: 0.11108104884624481
Epoch: 12, Loss: 4.339807510375977
Hits@1: 0.11474116146564484
Epoch: 13, Loss: 4.3158698081970215
Hits@1: 0.11943843215703964
Epoch: 14, Loss: 4.293863773345947
Hits@1: 0.12219666689634323
Epoch: 15, Loss: 4.273504734039307
Hits@1: 0.12543590366840363
E

### Edge dropout, only infomax

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=edge_dropout).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [ ]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_only_infomax(epoch)
  else:
    loss = train_only_infomax(epoch)
  loss_epoch.append(loss)
test_pos_z, test_neg_z, test_summary = model(dataset.test_features, dataset.edge_index)

  1%|          | 1/100 [00:08<14:41,  8.90s/it]

Epoch: 1, Loss: 1.5315818786621094


  2%|▏         | 2/100 [00:14<11:28,  7.03s/it]

Epoch: 2, Loss: 1.4307504892349243


  3%|▎         | 3/100 [00:21<11:01,  6.82s/it]

...

Epoch: 98, Loss: 0.5901374220848083


 99%|█████████▉| 99/100 [10:18<00:06,  6.15s/it]

Epoch: 99, Loss: 0.5753174424171448


100%|██████████| 100/100 [10:24<00:00,  6.25s/it]

Epoch: 100, Loss: 0.5405212640762329


In [ ]:
train_user_representations = {}
test_user_representations = {}
for i in range(pos_z.shape[0]):
  train_user_representations[i] = (dataset.user_ids[i], pos_z[i].detach(), dataset.train_labels[i])
for i in range(test_pos_z.shape[0]):
  test_user_representations[i] = (dataset.user_ids[i], test_pos_z[i].detach(), dataset.test_labels[i])

In [ ]:
user_dataset = UserDataset(train_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [ ]:
num_classes = 186
classes = np.arange(0, num_classes)

In [ ]:
head_model = train2task()
torch.save(head_model.state_dict(), 'edge_dropout_only_infomax_model.pth')

Epoch: 0, Loss: 4.957077503204346
Hits@1: 0.09370490163564682
Epoch: 1, Loss: 4.774438858032227
Hits@1: 0.09492995589971542
Epoch: 2, Loss: 4.651824951171875
Hits@1: 0.09392286092042923
Epoch: 3, Loss: 4.566333770751953
Hits@1: 0.09395292401313782
Epoch: 4, Loss: 4.503297328948975
Hits@1: 0.09492995589971542
Epoch: 5, Loss: 4.454209804534912
Hits@1: 0.09891323745250702
Epoch: 6, Loss: 4.414301872253418
Hits@1: 0.100679412484169
Epoch: 7, Loss: 4.380742073059082
Hits@1: 0.10337752103805542
Epoch: 8, Loss: 4.351809978485107
Hits@1: 0.10560967028141022
Epoch: 9, Loss: 4.326364517211914
Hits@1: 0.1086159199476242
Epoch: 10, Loss: 4.303642272949219
Hits@1: 0.10935996472835541
Epoch: 11, Loss: 4.283111572265625
Hits@1: 0.11449314653873444
Epoch: 12, Loss: 4.264362812042236
Hits@1: 0.11751443147659302
Epoch: 13, Loss: 4.247109889984131
Hits@1: 0.11849146336317062
Epoch: 14, Loss: 4.231112957000732
Hits@1: 0.12124969810247421
Epoch: 15, Loss: 4.216197490692139
Hits@1: 0.12369979172945023
Epoch

### Random walk, only infomax

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=random_walk).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [ ]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_only_infomax(epoch)
  else:
    loss = train_only_infomax(epoch)
  loss_epoch.append(loss)
test_pos_z, test_neg_z, test_summary = model(dataset.test_features, dataset.edge_index)

  1%|          | 1/100 [00:05<08:20,  5.06s/it]

Epoch: 1, Loss: 1.3601748943328857


  2%|▏         | 2/100 [00:08<06:45,  4.14s/it]

Epoch: 2, Loss: 1.3325636386871338


  3%|▎         | 3/100 [00:13<07:06,  4.40s/it]

...

Epoch: 98, Loss: 0.664185643196106


 99%|█████████▉| 99/100 [06:25<00:03,  3.96s/it]

Epoch: 99, Loss: 0.4624948799610138


100%|██████████| 100/100 [06:29<00:00,  3.89s/it]

Epoch: 100, Loss: 0.7446961402893066


In [ ]:
train_user_representations = {}
test_user_representations = {}
for i in range(pos_z.shape[0]):
  train_user_representations[i] = (dataset.user_ids[i], pos_z[i].detach(), dataset.train_labels[i])
for i in range(test_pos_z.shape[0]):
  test_user_representations[i] = (dataset.user_ids[i], test_pos_z[i].detach(), dataset.test_labels[i])

In [ ]:
user_dataset = UserDataset(train_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [ ]:
num_classes = 186
classes = np.arange(0, num_classes)

In [ ]:
head_model = train2task()
torch.save(head_model.state_dict(), 'random_walk_only_infomax_model.pth')

Epoch: 0, Loss: 5.0268049240112305
Hits@1: 0.038923460990190506
Epoch: 1, Loss: 4.866142272949219
Hits@1: 0.05279731750488281
Epoch: 2, Loss: 4.744835376739502
Hits@1: 0.08801557123661041
Epoch: 3, Loss: 4.6520304679870605
Hits@1: 0.10407647490501404
Epoch: 4, Loss: 4.579610347747803
Hits@1: 0.10286646336317062
Epoch: 5, Loss: 4.521663188934326
Hits@1: 0.10514369606971741
Epoch: 6, Loss: 4.474079608917236
Hits@1: 0.10661676526069641
Epoch: 7, Loss: 4.434088230133057
Hits@1: 0.10835286974906921
Epoch: 8, Loss: 4.399752140045166
Hits@1: 0.11255411058664322
Epoch: 9, Loss: 4.369768142700195
Hits@1: 0.11624428629875183
Epoch: 10, Loss: 4.343201160430908
Hits@1: 0.12146764993667603
Epoch: 11, Loss: 4.319370269775391
Hits@1: 0.12447390705347061
Epoch: 12, Loss: 4.297772407531738
Hits@1: 0.126473069190979
Epoch: 13, Loss: 4.278028964996338
Hits@1: 0.13014820218086243
Epoch: 14, Loss: 4.2598443031311035
Hits@1: 0.13210226595401764
Epoch: 15, Loss: 4.242992401123047
Hits@1: 0.13332732021808624


### Node mix, Infomax + Link pred

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=node_mix).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [ ]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_infomax_linkpred(epoch)
  else:
    loss = train_infomax_linkpred(epoch)
  loss_epoch.append(loss)
test_pos_z, test_neg_z, test_summary = model(dataset.test_features, dataset.edge_index)

  1%|          | 1/100 [00:10<16:33, 10.03s/it]

Epoch: 1, Loss: 4.500741958618164


  2%|▏         | 2/100 [00:17<13:39,  8.36s/it]

Epoch: 2, Loss: 3.718627452850342


  3%|▎         | 3/100 [00:25<13:23,  8.29s/it]

...

Epoch: 98, Loss: 0.9257211685180664


 99%|█████████▉| 99/100 [12:26<00:07,  7.87s/it]

Epoch: 99, Loss: 0.6998384594917297


100%|██████████| 100/100 [12:33<00:00,  7.53s/it]

Epoch: 100, Loss: 0.7707847356796265


In [ ]:
train_user_representations = {}
test_user_representations = {}
for i in range(pos_z.shape[0]):
  train_user_representations[i] = (dataset.user_ids[i], pos_z[i].detach(), dataset.train_labels[i])
for i in range(test_pos_z.shape[0]):
  test_user_representations[i] = (dataset.user_ids[i], test_pos_z[i].detach(), dataset.test_labels[i])

In [ ]:
user_dataset = UserDataset(train_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [ ]:
num_classes = 186
classes = np.arange(0, num_classes)

In [ ]:
head_model = train2task()
torch.save(head_model.state_dict(), 'node_mix_infomax_linkpred_model.pth')

Epoch: 0, Loss: 5.155427932739258
Hits@1: 0.06047077849507332
Epoch: 1, Loss: 5.084137916564941
Hits@1: 0.08645232021808624
Epoch: 2, Loss: 5.018618583679199
Hits@1: 0.10460256785154343
Epoch: 3, Loss: 4.958208084106445
Hits@1: 0.12050565332174301
Epoch: 4, Loss: 4.902405261993408
Hits@1: 0.12737493216991425
Epoch: 5, Loss: 4.850795745849609
Hits@1: 0.12934403121471405
Epoch: 6, Loss: 4.803010940551758
Hits@1: 0.13211730122566223
Epoch: 7, Loss: 4.75872278213501
Hits@1: 0.13189934194087982
Epoch: 8, Loss: 4.717621326446533
Hits@1: 0.13388347625732422
Epoch: 9, Loss: 4.679427146911621
Hits@1: 0.13461248576641083
Epoch: 10, Loss: 4.643889427185059
Hits@1: 0.13459746539592743
Epoch: 11, Loss: 4.610775947570801
Hits@1: 0.13484548032283783
Epoch: 12, Loss: 4.579861640930176
Hits@1: 0.13509349524974823
Epoch: 13, Loss: 4.550949573516846
Hits@1: 0.13658158481121063
Epoch: 14, Loss: 4.523855209350586
Hits@1: 0.13707761466503143
Epoch: 15, Loss: 4.4984130859375
Hits@1: 0.13709264993667603
Epoch

### Node dropout, Infomax + Link pred

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=node_dropout).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [ ]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_infomax_linkpred(epoch)
  else:
    loss = train_infomax_linkpred(epoch)
  loss_epoch.append(loss)
test_pos_z, test_neg_z, test_summary = model(dataset.test_features, dataset.edge_index)

  1%|          | 1/100 [00:10<16:50, 10.20s/it]

Epoch: 1, Loss: 3.8799002170562744


  2%|▏         | 2/100 [00:17<14:05,  8.63s/it]

Epoch: 2, Loss: 3.5561585426330566


  3%|▎         | 3/100 [00:27<14:44,  9.12s/it]

...

Epoch: 98, Loss: 1.912804365158081


 99%|█████████▉| 99/100 [13:47<00:08,  8.13s/it]

Epoch: 99, Loss: 2.6845946311950684


100%|██████████| 100/100 [13:54<00:00,  8.35s/it]

Epoch: 100, Loss: 1.856714129447937


In [ ]:
train_user_representations = {}
test_user_representations = {}
for i in range(pos_z.shape[0]):
  train_user_representations[i] = (dataset.user_ids[i], pos_z[i].detach(), dataset.train_labels[i])
for i in range(test_pos_z.shape[0]):
  test_user_representations[i] = (dataset.user_ids[i], test_pos_z[i].detach(), dataset.test_labels[i])

In [ ]:
user_dataset = UserDataset(train_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [ ]:
num_classes = 186
classes = np.arange(0, num_classes)

In [ ]:
head_model = train2task()
torch.save(head_model.state_dict(), 'node_dropout_infomax_linkpred_model.pth')

Epoch: 0, Loss: 5.17767858505249
Hits@1: 0.057772666215896606
Epoch: 1, Loss: 5.126504898071289
Hits@1: 0.09021764993667603
Epoch: 2, Loss: 5.078027248382568
Hits@1: 0.10703763365745544
Epoch: 3, Loss: 5.031926155090332
Hits@1: 0.11799542605876923
Epoch: 4, Loss: 4.9880218505859375
Hits@1: 0.13461248576641083
Epoch: 5, Loss: 4.946174621582031
Hits@1: 0.144578218460083
Epoch: 6, Loss: 4.906243801116943
Hits@1: 0.15295062959194183
Epoch: 7, Loss: 4.868104934692383
Hits@1: 0.15837691724300385
Epoch: 8, Loss: 4.831657886505127
Hits@1: 0.15811386704444885
Epoch: 9, Loss: 4.796802520751953
Hits@1: 0.15986502170562744
Epoch: 10, Loss: 4.7634501457214355
Hits@1: 0.15988004207611084
Epoch: 11, Loss: 4.731516361236572
Hits@1: 0.16087211668491364
Epoch: 12, Loss: 4.700922966003418
Hits@1: 0.16333724558353424
Epoch: 13, Loss: 4.671606540679932
Hits@1: 0.1624203324317932
Epoch: 14, Loss: 4.643500804901123
Hits@1: 0.16487042605876923
Epoch: 15, Loss: 4.61654806137085
Hits@1: 0.16435936093330383
Epoc

### Edge dropout, Infomax + Link pred

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=edge_dropout).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [ ]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_infomax_linkpred(epoch)
  else:
    loss = train_infomax_linkpred(epoch)
  loss_epoch.append(loss)
test_pos_z, test_neg_z, test_summary = model(dataset.test_features, dataset.edge_index)

  1%|          | 1/100 [00:08<13:23,  8.11s/it]

Epoch: 1, Loss: 7.972873687744141


  2%|▏         | 2/100 [00:15<12:37,  7.73s/it]

Epoch: 2, Loss: 6.6683197021484375


  3%|▎         | 3/100 [00:21<11:04,  6.86s/it]

...

Epoch: 98, Loss: 1.9780542850494385


 99%|█████████▉| 99/100 [10:39<00:06,  6.52s/it]

Epoch: 99, Loss: 1.9169942140579224


100%|██████████| 100/100 [10:45<00:00,  6.45s/it]

Epoch: 100, Loss: 2.0916476249694824


In [ ]:
train_user_representations = {}
test_user_representations = {}
for i in range(pos_z.shape[0]):
  train_user_representations[i] = (dataset.user_ids[i], pos_z[i].detach(), dataset.train_labels[i])
for i in range(test_pos_z.shape[0]):
  test_user_representations[i] = (dataset.user_ids[i], test_pos_z[i].detach(), dataset.test_labels[i])

In [ ]:
user_dataset = UserDataset(train_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [ ]:
num_classes = 186
classes = np.arange(0, num_classes)

In [ ]:
head_model = train2task()
torch.save(head_model.state_dict(), 'edge_dropout_infomax_linkpred_model.pth')

Epoch: 0, Loss: 5.160297393798828
Hits@1: 0.05130922049283981
Epoch: 1, Loss: 5.097674369812012
Hits@1: 0.072420634329319
Epoch: 2, Loss: 5.039208889007568
Hits@1: 0.10237042605876923
Epoch: 3, Loss: 4.984537601470947
Hits@1: 0.11653739959001541
Epoch: 4, Loss: 4.9333577156066895
Hits@1: 0.12617994844913483
Epoch: 5, Loss: 4.885403156280518
Hits@1: 0.13459746539592743
Epoch: 6, Loss: 4.840405464172363
Hits@1: 0.14278198778629303
Epoch: 7, Loss: 4.798115253448486
Hits@1: 0.14825336635112762
Epoch: 8, Loss: 4.758296966552734
Hits@1: 0.15389759838581085
Epoch: 9, Loss: 4.720736026763916
Hits@1: 0.16037608683109283
Epoch: 10, Loss: 4.685244083404541
Hits@1: 0.16354015469551086
Epoch: 11, Loss: 4.651653289794922
Hits@1: 0.16536645591259003
Epoch: 12, Loss: 4.619807720184326
Hits@1: 0.17178480327129364
Epoch: 13, Loss: 4.589570999145508
Hits@1: 0.17503908276557922
Epoch: 14, Loss: 4.560819625854492
Hits@1: 0.17753426730632782
Epoch: 15, Loss: 4.533444881439209
Hits@1: 0.18020232021808624
Epo

### Random walk, Infomax + Link pred

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=random_walk).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [ ]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_infomax_linkpred(epoch)
  else:
    loss = train_infomax_linkpred(epoch)
  loss_epoch.append(loss)
test_pos_z, test_neg_z, test_summary = model(dataset.test_features, dataset.edge_index)

  1%|          | 1/100 [00:05<09:24,  5.70s/it]

Epoch: 1, Loss: 4.511188983917236


  2%|▏         | 2/100 [00:10<08:10,  5.01s/it]

Epoch: 2, Loss: 3.2485270500183105


  3%|▎         | 3/100 [00:14<07:17,  4.51s/it]

...

Epoch: 98, Loss: 1.7433596849441528


 99%|█████████▉| 99/100 [07:02<00:04,  4.29s/it]

Epoch: 99, Loss: 1.2765605449676514


100%|██████████| 100/100 [07:06<00:00,  4.26s/it]

Epoch: 100, Loss: 1.296288013458252


In [ ]:
train_user_representations = {}
test_user_representations = {}
for i in range(pos_z.shape[0]):
  train_user_representations[i] = (dataset.user_ids[i], pos_z[i].detach(), dataset.train_labels[i])
for i in range(test_pos_z.shape[0]):
  test_user_representations[i] = (dataset.user_ids[i], test_pos_z[i].detach(), dataset.test_labels[i])

In [ ]:
user_dataset = UserDataset(train_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [ ]:
num_classes = 186
classes = np.arange(0, num_classes)

In [ ]:
head_model = train2task()
torch.save(head_model.state_dict(), 'random_walk_infomax_linkpred_model.pth')

Epoch: 0, Loss: 5.178340911865234
Hits@1: 0.04955808073282242
Epoch: 1, Loss: 5.1121368408203125
Hits@1: 0.07634379714727402
Epoch: 2, Loss: 5.050725936889648
Hits@1: 0.09246482700109482
Epoch: 3, Loss: 4.993563175201416
Hits@1: 0.10117544233798981
Epoch: 4, Loss: 4.940242767333984
Hits@1: 0.11373406648635864
Epoch: 5, Loss: 4.890415191650391
Hits@1: 0.12101671099662781
Epoch: 6, Loss: 4.843774318695068
Hits@1: 0.12841209769248962
Epoch: 7, Loss: 4.800050735473633
Hits@1: 0.13436447083950043
Epoch: 8, Loss: 4.759006500244141
Hits@1: 0.1408730149269104
Epoch: 9, Loss: 4.720427989959717
Hits@1: 0.14826840162277222
Epoch: 10, Loss: 4.684117794036865
Hits@1: 0.15346169471740723
Epoch: 11, Loss: 4.649898052215576
Hits@1: 0.15693391859531403
Epoch: 12, Loss: 4.61760950088501
Hits@1: 0.15960197150707245
Epoch: 13, Loss: 4.587100505828857
Hits@1: 0.16236020624637604
Epoch: 14, Loss: 4.558238506317139
Hits@1: 0.16190926730632782
Epoch: 15, Loss: 4.530893325805664
Hits@1: 0.16310425102710724
Epo

## Training on target experiments

### Concatenation

In [ ]:
users_list = load_dataset_timestamp(20001, 128, 30)

In [ ]:
class UserInfoDataset():
  def __init__(self, data, max_len, train_user_representations, test_user_representations):
    self.data = data
    self.max_len = max_len
    self.train_user_representations = train_user_representations
    self.test_user_representations = test_user_representations

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):

    user = self.data[idx]
    seq_len = user['seq_len']

    tr_act_seq = np.zeros((self.max_len,)).astype('int32')
    tr_act_seq[:seq_len] = np.array(user['train_act_seq'])
    tr_act_seq = np.transpose(tr_act_seq)

    tr_time_seq = np.zeros((self.max_len,)).astype('int32')
    tr_time_seq[:seq_len] = user['train_time_seq']
    tr_time_seq = np.transpose(tr_time_seq)

    t_act_seq = np.zeros((self.max_len, )).astype('int32')
    t_act_seq[:seq_len] = user['test_act_seq']
    t_act_seq = np.transpose(t_act_seq)

    t_time_seq = np.zeros((self.max_len, )).astype('int32')
    t_time_seq[:seq_len] = user['test_time_seq']
    t_time_seq = np.transpose(t_time_seq)

    if idx not in self.train_user_representations.keys():
      self.train_user_representations[idx] = (torch.zeros(size=(128,)), torch.tensor(0))
      self.test_user_representations[idx] = (torch.zeros(size=(128,)), torch.tensor(0))


    return user['user'], tr_act_seq, \
    tr_time_seq, user['train_act_label'], \
    user['train_time_label'], t_act_seq, \
    t_time_seq, user['test_act_label'], \
    user['test_time_label'], user['seq_len'], self.train_user_representations[idx][1], self.test_user_representations[idx][1]

In [ ]:
user_dataset = UserInfoDataset(users_list, 30, train_user_representations, test_user_representations)
user_dataloader = DataLoader(user_dataset, batch_size = 64, shuffle=True)

Concatenation with pre-trained embedding

In [ ]:
class RNNTargetModel(nn.Module):
  def __init__(self, num_classes):
    super(RNNTargetModel, self).__init__()
    self.fc1 = nn.Linear(256, num_classes)
    self.rnn = nn.RNN(128, 128, batch_first = True)
    self.norm = nn.BatchNorm1d(128)
    self.reduce_dim = nn.Linear(128, 128)

  def forward(self, x, repr, seq_len):
    x, h = self.rnn(x)
    hx = torch.zeros(x.shape[0], x.shape[2])
    for i in range(hx.shape[0]):
      hx[i] = x[i][seq_len[i] - 1]

    hx = torch.cat([hx, repr], dim= 1)

    x = self.fc1(hx)
    return x

Concatenation with random embedding

In [ ]:
class RNNTargetModel(nn.Module):
  def __init__(self, num_classes):
    super(RNNTargetModel, self).__init__()
    self.fc1 = nn.Linear(256, num_classes)
    self.rnn = nn.RNN(128, 128, batch_first = True)
    self.norm = nn.BatchNorm1d(128)
    self.reduce_dim = nn.Linear(128, 128)

  def forward(self, x, repr, seq_len):
    x, h = self.rnn(x)
    hx = torch.zeros(x.shape[0], x.shape[2])
    for i in range(hx.shape[0]):
      hx[i] = x[i][seq_len[i] - 1]

    repr = torch.rand((len(x), 128))
    hx = torch.cat([hx, repr], dim= 1)
    x = self.fc1(hx)
    return x

Without embedding add

In [ ]:
class RNNTargetModel(nn.Module):
  def __init__(self, num_classes):
    super(RNNTargetModel, self).__init__()
    self.fc1 = nn.Linear(256, num_classes)
    self.rnn = nn.RNN(128, 128, batch_first = True)
    self.norm = nn.BatchNorm1d(128)
    self.reduce_dim = nn.Linear(128, 128)

  def forward(self, x, repr, seq_len):
    x, h = self.rnn(x)
    hx = torch.zeros(x.shape[0], x.shape[2])
    for i in range(hx.shape[0]):
      hx[i] = x[i][seq_len[i] - 1]
    x = self.fc1(hx)
    return x

In [ ]:
def rnn_target_test(model):
  metrics_val = []
  model.eval()
  index = 0

  for user, train_input, train_time, train_label, train_time_label, test_input, test_time, test_label, test_time_label, seq_len, train_repr, test_repr in user_dataloader:

      test_comb_input = np.concatenate([np.expand_dims(test_input, axis=-1),
                                                np.expand_dims(test_time, axis=-1)], axis=2)
      model_input = test_comb_input
      model_output = test_label
      test_rnn_input_emb = item_emb[model_input[:, :, 0]]
      test_probs = model(test_rnn_input_emb, test_repr, seq_len)

      test_pred = torch.argmax(test_probs, axis = 1)

      test_one_hot = torch.zeros(len(test_probs), num_classes)
      test_one_hot[torch.arange(len(test_one_hot)), test_label] = 1
      loss = loss_fn(test_probs, test_one_hot)

      hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, \
      ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = get_metrics_(test_probs, test_label, test_one_hot)

      metrics_val.append([hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100])


  mean = torch.Tensor(metrics_val).mean(axis=0)
  test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, \
  test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100 = mean
  return test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100,\
  test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100


In [ ]:
model = RNNTargetModel(num_classes)
loss_fn = nn.CrossEntropyLoss()
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.97, last_epoch=-1)


for i in range(20):
  losses = []
  val_losses = []
  hits_1_scores = []

  for user, train_input, train_time, train_label, train_time_label, test_input, test_time, test_label, \
  test_time_label, seq_len, train_repr, test_repr in user_dataloader:
      optimizer.zero_grad()
      comb_input = np.concatenate([np.expand_dims(train_input, axis=-1),
                                                np.expand_dims(train_time, axis=-1)], axis=2)
      model_input = comb_input
      model_output = train_label
      rnn_input_emb = item_emb[model_input[:, :, 0]]
      test_comb_input = np.concatenate([np.expand_dims(test_input, axis=-1),
                                                np.expand_dims(test_time, axis=-1)], axis=2)
      test_model_input = test_comb_input
      test_rnn_input_emb = item_emb[test_model_input[:, :, 0]]
      probs = model(rnn_input_emb, train_repr, seq_len)
      test_probs = model(test_rnn_input_emb, test_repr, seq_len)
      pred = torch.argmax(probs, axis = 1)

      one_hot = torch.zeros(len(probs), num_classes)
      one_hot[torch.arange(len(one_hot)), model_output] = 1

      test_one_hot = torch.zeros(len(test_probs), num_classes)
      test_one_hot[torch.arange(len(test_one_hot)), test_label] = 1

      loss = loss_fn(probs, one_hot)
      val_loss = loss_fn(test_probs, test_one_hot)
      losses.append(loss)
      val_losses.append(val_loss)
      hits_1_scores.append(top_k_accuracy_score(test_label, test_probs.cpu().detach().numpy(), k=1, labels = classes))
      loss.backward()
      optimizer.step()
      losses.append(loss)


  mean_loss = torch.Tensor(losses).mean(axis=0)
  mean_val_loss = torch.Tensor(val_losses).mean(axis = 0).item()
  mean_hits = torch.Tensor(hits_1_scores).mean(axis=0).item()
  print(f'Epoch: {i} Loss: {mean_loss.item()}, Val loss: {mean_val_loss} Hits@1: {mean_hits}')
hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = rnn_target_test(model)
print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100)

Epoch: 0 Loss: 4.462273597717285, Val loss: 4.467819690704346 Hits@1: 0.06983523815870285
Epoch: 1 Loss: 4.01273775100708, Val loss: 4.0267839431762695 Hits@1: 0.13592343032360077
Epoch: 2 Loss: 3.4965574741363525, Val loss: 3.5176172256469727 Hits@1: 0.25534600019454956
Epoch: 3 Loss: 2.9872641563415527, Val loss: 3.0122857093811035 Hits@1: 0.3300824761390686
Epoch: 4 Loss: 2.592750310897827, Val loss: 2.621905565261841 Hits@1: 0.4544183909893036
Epoch: 5 Loss: 2.2767646312713623, Val loss: 2.3081257343292236 Hits@1: 0.5460777878761292
Epoch: 6 Loss: 2.023632049560547, Val loss: 2.0590434074401855 Hits@1: 0.6054298281669617
Epoch: 7 Loss: 1.8238966464996338, Val loss: 1.8630046844482422 Hits@1: 0.6465548872947693
Epoch: 8 Loss: 1.6682329177856445, Val loss: 1.712514042854309 Hits@1: 0.677602231502533
Epoch: 9 Loss: 1.5491427183151245, Val loss: 1.5955711603164673 Hits@1: 0.6961286067962646
Epoch: 10 Loss: 1.4537062644958496, Val loss: 1.5024302005767822 Hits@1: 0.7095329165458679
Epoc